# Nonlinear Optimization

Alberto Quaini

Import libraries

In [14]:
import numpy as np
from scipy.linalg import norm
from scipy import optimize as opt

## Exercise 6

In [63]:
def quad_steepestDescent(Q, b, c, x_0, eps=1e-9, max_iter=1e5):
    """ Solve quadratic optimization problem of the form
    x^TQx -b^Tx+c, where Q is an nxn positive definite matrix,
    b, x are vectors in R^n and c is a real number.    
    Method: Steepest descent.
    """
    n = np.size(x_0)
    if not np.size(b) == n:
        raise ValueError('b is not of length n (n being the size of x_0)')
    if not Q.shape == (n,n):
        raise ValueError('Q is not a nxn matrix, (n being the size of x_0)')
    
    x1 = x_0
    criterion = eps+1
    for i in range(int(max_iter)):
        
        M = Q @ x_0 - b
        MT = M.T
        a_num = MT @ M
        a_den = MT @ Q @ M
        a = a_num / a_den
        x_1 = x_0 - a * M
        criterion = norm(MT)
        if (criterion < eps):
            break
        x_0 = x_1
    
    f = 0.5 * x_1.T @ Q @ x_1 - b.T @ x_1 + c
    if (i < max_iter):
        print('Converged')
    else:
        print('Did not converge!')
    return x_1, f
    

Test function

In [64]:
Q = np.array([[3, 1],[1, 1.5]])
b = np.array([-2, 1])
c = 0
x_0 = np.array([1.1, 1.5])
quad_steepestDescent(Q, b, c, x_0)

Converged


(array([-1.14285714,  1.42857143]), -1.8571428571428568)

In [65]:
def quad(x, args):
    Q, b, c = args
    return 0.5 * x.T @ Q @ x - b.T @ x + c

In [66]:
opt.minimize(quad, x_0, args=[Q, b, c])

      fun: -1.8571428571428568
 hess_inv: array([[ 0.42886341, -0.28561563],
       [-0.28561563,  0.85717585]])
      jac: array([5.96046448e-08, 2.98023224e-08])
  message: 'Optimization terminated successfully.'
     nfev: 20
      nit: 4
     njev: 5
   status: 0
  success: True
        x: array([-1.14285713,  1.42857143])

## Exercise 7 

In [107]:
def num_gradient(f, x, rerr=1e-8):
    """ Compute the numerical gradient of function
    f:R^n->R, at a given point x in R^n.
    rerr is an estimate for the maximum relative error
    of f near x.
    """
    
    n = np.size(x)
    h = np.sqrt(rerr)
    grad = np.zeros(n)
    
    for i in range(n):
        
        ei = np.eye(n)[i]
        grad[i] = (f(x + h*ei) - f(x - h*ei)) / (2*h)
    
    return grad

In [108]:
f = lambda x: np.array([x[0]**2 + 4*x[1] - x[2]])
x = np.array([1, 2, 3])
num_gradient(f, x)

array([ 2.,  4., -1.])

## Exercise 8

In [139]:
def secant_minimizer(x0, x1, f_prime, eps=1e-9):
    """ Find the minimizer of a function 
    via the secant method.
    Initial values: x0 and x1,
    Accuracy: eps,
    First derivative of f: f_prime.
    """
    max_iter = int(1e4)
    dist = eps + 1
    xvec = [x0, x1]
    
    for i in range(max_iter):
        x = x1 - f_prime(x1) * (x1-x0) / ( f_prime(x1)-f_prime(x0) )
        x0 = x1
        x1 = x
        xvec.append(x1)
        dist = np.abs(x1 - x0) #/ np.abs(x0)
        try:
            if (dist < eps):
                break
        except:
            print(x)
            break
    
    if i == max_iter:
        print('Secant method did not converge')
        
    return xvec

In [143]:
def steepestDescent(f, x_0, eps=1e-9, max_iter=1e5):
    """ Solve unconstrained nonlinear optimization problems
    via the steepest descent method.
    """
    n = np.size(x_0)
    grad_f = lambda x: num_gradient(f, x)
    f_secant = lambda a, x: grad_f( x - a*grad_f(x) ).T @ grad_f(x)
    
    for i in range(int(max_iter)):
      
        grad = grad_f(x_0)
        f_sec = lambda a: f_secant(a, x_0)
        a = secant_minimizer(1, 1.5, f_sec, eps)[-1]
        x_1 = x_0 - a * grad
        criterion = norm(x_0 - x_1)
        if (criterion < eps):
            break
        x_0 = x_1
    
    if (i < max_iter):
        print('Steepest descent routine converged')
    else:
        print('Steepest descent routine did not converge')
        
    return x_1

In [148]:
def rosenbrock(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2

In [149]:
xvec_guess = np.array([-2,2])
xmin = steepestDescent(rosenbrock, xvec_guess)
print('minimizer is: ', xmin)
print('minimum is: ', rosenbrock(xmin))

Steepest descent routine converged
minimizer is:  [0.99999758 0.99999516]
minimum is:  5.855454170821065e-12


YESSS!!!